In [1]:
import functools
from dataclasses import dataclass


def load_input() -> list[list[str]]:
    with open("../../data/day12-input.txt") as f:
        grid = []
        for line in f.readlines():
            line = line.strip()
            row = []
            for char in line:
                row.append(char)
            grid.append(row)
    return grid


@functools.lru_cache(maxsize=None)
def move_to(row, col, direction) -> tuple[int, int]:
    dirs = {
        'up': lambda i, j: (i - 1, j),
        'down': lambda i, j: (i + 1, j),
        'left': lambda i, j: (i, j - 1),
        'right': lambda i, j: (i, j + 1),
    }
    return dirs[direction](row, col)



In [2]:
@dataclass()
class GridSquare:
    row: int
    col: int
    value: str
    marked: bool = False

    def __hash__(self) -> int:
        return hash((self.row, self.col, self.value))


In [3]:
def create_grid():
    raw_grid = load_input()
    grid = []
    for i, row in enumerate(raw_grid):
        grid_row = []
        for j, col in enumerate(row):
            grid_row.append(GridSquare(i, j, col, False))
        grid.append(grid_row)
    return grid


def clear_grid_markers(grid: list[list[GridSquare]]) -> None:
    for row in grid:
        for square in row:
            square.marked = False


In [6]:
def collect_region(current_square: GridSquare, value_to_match: str, grid: list[list[GridSquare]]) -> None:
    collection = set()
    if current_square.value != value_to_match:
        return collection
    else:
        current_square.marked = True
        collection.add(current_square)
        for direction in ('up', 'down', 'left', 'right'):
            next_row, next_col = move_to(current_square.row, current_square.col, direction)
            if -1 < next_row < len(grid) and -1 < next_col < len(grid[0]):
                next_square = grid[next_row][next_col]
                if next_square.marked:
                    continue
                else:
                    collection = collection | collect_region(next_square, value_to_match, grid)
    return collection


def collect_regions(grid: list[list[GridSquare]]) -> list[set[GridSquare]]:
    regions = []
    for i, row in enumerate(grid):
        for j, square in enumerate(row):
            if square.marked:
                continue
            region = collect_region(square, square.value, grid)
            if region:
                regions.append(region)
    return regions

def count_borders(square: GridSquare, grid: list[list[GridSquare]]) -> int:
    borders = 0
    for direction in ('up', 'down', 'left', 'right'):
        next_row, next_col = move_to(square.row, square.col, direction)
        if -1 < next_row < len(grid) and -1 < next_col < len(grid[0]):
            next_square = grid[next_row][next_col]
            if next_square.value != square.value:
                borders += 1
        else:
            borders += 1
    return borders



def calculate_cost(region: set[GridSquare], grid: list[list[GridSquare]]) -> int:
    perimeter = 0
    area = 0
    for square in region:
        area += 1
        perimeter += count_borders(square, grid)
    return area * perimeter


grid = create_grid()
regions = collect_regions(grid)
clear_grid_markers(regions)
total_cost = 0
for region in regions:
    total_cost += calculate_cost(region, grid)
print(total_cost)

1375476


### Part 2